In [1]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import requests

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [2]:
#we need to use a user agent here, because metacritics does not accept bots
#we are using a tested go-to user agent here, it can be found online
#read more about user agents here:
#https://www.scrapehero.com/how-to-fake-and-rotate-user-agents-using-python-3/
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

WEB SCRAPING FOR METACRITIC

In [3]:
url = "https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc"
page = requests.get(url, headers = headers)
soup = BeautifulSoup(page.content, "html.parser")

In [4]:
container = soup.find_all('td', class_ = 'clamp-summary-wrap')

In [5]:
#Finding the movie names through tag h3 and appending them movie_names list
movie_names = []
movies = container[:50] #here we get the top 50 movies we want
for movie in movies:
    name = movie.find('h3').text
    movie_names.append(name)

In [6]:
#now we need to click into each movie name in order to see detailed information
#so we need to extract all the urls
movie_links = []
movies = container[:50]
for movie in movies:
    tag = movie.find('a', class_ = 'title')
    link = tag.get('href', None)
    movie_links.append(link)

In [7]:
for i in range(len(movie_links)):
    movie_links[i] = 'https://www.metacritic.com'+ movie_links[i]

In [8]:
#there are a few different ways to get the links to genres
#lets first try to write a loop to open all 50 links of these movies
#Looping through all the links and finding the 'div' tag with class 'genres'
#Decomposing all the 'span' tags to find the content inside and finding the movie name from splitting the previous movie links 
#Replacing all the <span> and </span> tags with space and printing the dictionary
genres_links = []
genres = []
metacritic_dict = dict()
for i in movie_links:
    page = requests.get(i, headers = headers)
    soup = BeautifulSoup(page.content, "html.parser")
    tag = soup.find('div', class_= 'genres')
    genres_link = tag.span.decompose()
    genres_link = tag.find_all('span', None)
    movname = i.split('/')[-1]
    templist = []
    for j in range(1,len(genres_link)):
        temp = str(genres_link[j]).replace('<span>', '').replace('</span>' , '')
        templist.append(temp)
    metacritic_dict[movname] = templist
print(metacritic_dict)

{'citizen-kane': ['Drama', 'Mystery'], 'the-godfather': ['Drama', 'Thriller', 'Crime'], 'rear-window': ['Mystery', 'Thriller'], 'casablanca': ['Drama', 'Romance', 'War'], 'boyhood': ['Drama'], 'three-colors-red': ['Drama', 'Mystery', 'Romance'], 'vertigo': ['Mystery', 'Thriller', 'Romance'], 'notorious-1946': ['Drama', 'Thriller', 'Romance', 'Film-Noir'], 'singin-in-the-rain': ['Comedy', 'Romance', 'Musical'], 'city-lights': ['Drama', 'Comedy', 'Romance'], 'moonlight-2016': ['Drama'], 'intolerance': ['Drama', 'History'], 'pinocchio-1940': ['Fantasy', 'Comedy', 'Animation', 'Family', 'Musical'], 'touch-of-evil': ['Drama', 'Thriller', 'Crime', 'Film-Noir'], 'the-treasure-of-the-sierra-madre': ['Adventure', 'Drama', 'Western'], 'pans-labyrinth': ['Drama', 'Mystery', 'Thriller', 'Fantasy', 'War'], 'some-like-it-hot': ['Comedy', 'Romance'], 'north-by-northwest': ['Action', 'Adventure', 'Mystery', 'Thriller'], 'hoop-dreams': ['Drama', 'Sport', 'Documentary'], 'rashomon': ['Drama', 'Mystery',

WEB SCRAPING FOR ROTTEN TOMATOES 

In [9]:
url2 = "https://www.rottentomatoes.com/top/bestofrt/"
page2 = requests.get(url2, headers = headers)
soup = BeautifulSoup(page2.content, "html.parser")

In [10]:

rottenlinks = []
for i in soup.find_all('a' , class_ = 'unstyled articleLink', href = True):
    #print(i['href'])
    rottenlinks.append(i['href'])
startindex = rottenlinks.index('/m/black_panther_2018')
endindex = rottenlinks.index('/m/chinatown')
finallinks = rottenlinks[startindex:endindex+1]
finalrottenlinks = finallinks[:50]

In [11]:
for j in range(len(finalrottenlinks)):
    finalrottenlinks[j] = 'https://www.rottentomatoes.com' + finalrottenlinks[j]

In [12]:
#Printing the Rotten Tomatoes dictionary
rotten_tomatoes_dict = dict()
for i in finalrottenlinks:
    page3 = requests.get(i, headers = headers)
    soup = BeautifulSoup(page3.content, "html.parser")
    tag = soup.find_all('div', class_ = 'meta-value genre')
    movname2 = i.split('/')[-1]
    for x in tag:
        gen = x.contents
        templist2 =[]
        for k in range(0,len(gen)):
            temp = str(gen[k]).replace('\n' , '').strip()
            #templist2.append(temp)
            temp = temp.replace(' ','')
            rotten_tomatoes_dict[movname2] = temp.split(',')
print(rotten_tomatoes_dict)        

{'black_panther_2018': ['fantasy', 'action', 'adventure'], 'avengers_endgame': ['fantasy', 'adventure', 'scifi', 'action'], 'us_2019': ['horror', 'mysteryandthriller'], 'toy_story_4': ['fantasy', 'kidsandfamily', 'animation', 'adventure', 'comedy'], 'lady_bird': ['drama', 'comedy'], 'citizen_kane': ['drama'], 'mission_impossible_fallout': ['adventure', 'mysteryandthriller', 'action'], 'the_wizard_of_oz_1939': ['musical', 'kidsandfamily', 'fantasy'], 'blackkklansman': ['drama', 'comedy', 'crime'], 'get_out': ['horror', 'mysteryandthriller', 'comedy'], 'the_irishman': ['drama', 'crime'], 'mad_max_fury_road': ['adventure', 'action'], 'spider_man_into_the_spider_verse': ['fantasy', 'comedy', 'kidsandfamily', 'animation', 'adventure', 'action'], '1003707-casablanca': ['drama'], 'moonlight_2016': ['drama'], 'a_star_is_born_2018': ['drama', 'music', 'romance'], 'wonder_woman_2017': ['adventure', 'action', 'fantasy'], 'dunkirk_2017': ['drama', 'war', 'history'], 'inside_out_2015': ['comedy', '

WEB SCRAPING FOR IMDB

In [13]:
url = 'https://www.imdb.com/search/title/?groups=top_250&sort=user_rating' 
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')

In [14]:
movies = soup.select('.lister-item-header a')

In [15]:
genre = soup.select('.genre')

In [16]:
movies_name = [title.text for title in movies]
genre_type = [gn.text for gn in genre]

In [17]:
genre_t = []
for i in genre_type:
    i = i.replace('\n','')
    i = i.replace('            ','')
    genre_t.append(i)

In [18]:
genre_list = []
for g in genre_t:
    g = g.split(',')
    genre_list.append(g)

In [19]:
dictionary_movie_genre = dict(zip(movies_name,genre_list))

In [20]:
#Printing the IMDB dictionary

dictionary_movie_genre

{'The Shawshank Redemption': ['Drama'],
 'The Godfather': ['Crime', ' Drama'],
 'The Dark Knight': ['Action', ' Crime', ' Drama'],
 'The Godfather: Part II': ['Crime', ' Drama'],
 '12 Angry Men': ['Crime', ' Drama'],
 'The Lord of the Rings: The Return of the King': ['Action',
  ' Adventure',
  ' Drama'],
 'Pulp Fiction': ['Crime', ' Drama'],
 "Schindler's List": ['Biography', ' Drama', ' History'],
 'Inception': ['Action', ' Adventure', ' Sci-Fi'],
 'Fight Club': ['Drama'],
 'The Lord of the Rings: The Fellowship of the Ring': ['Action',
  ' Adventure',
  ' Drama'],
 'Forrest Gump': ['Drama', ' Romance'],
 'The Good, the Bad and the Ugly': ['Western'],
 'The Lord of the Rings: The Two Towers': ['Action', ' Adventure', ' Drama'],
 'The Matrix': ['Action', ' Sci-Fi'],
 'Goodfellas': ['Biography', ' Crime', ' Drama'],
 'Star Wars: Episode V - The Empire Strikes Back': ['Action',
  ' Adventure',
  ' Fantasy'],
 "One Flew Over the Cuckoo's Nest": ['Drama'],
 'Hamilton': ['Biography', ' Dra

In [22]:
#Counting occurrences of each genre of metacritic_dict and converting them into lowercase letters and importing them into another dictionary
from collections import OrderedDict 
l1 = list(metacritic_dict.values())
flattened  = [val for sublist in l1 for val in sublist]
dict1 = {}
for x in flattened:
    if x in dict1:
        dict1[x]+= 1
    else:
        dict1[x] = 1
        
dict1_lower = {k.lower().strip(): v for k, v in dict1.items()}   
print(dict1_lower)


{'drama': 39, 'mystery': 10, 'thriller': 13, 'crime': 4, 'romance': 11, 'war': 5, 'film-noir': 4, 'comedy': 9, 'musical': 3, 'history': 5, 'fantasy': 3, 'animation': 3, 'family': 3, 'adventure': 3, 'western': 2, 'action': 3, 'sport': 1, 'documentary': 1, 'biography': 2, 'horror': 3, 'music': 1}


In [23]:
#Counting occurrences of each genre of rotten_tomatoes_dict and converting them into lowercase letters 
l2 = list(rotten_tomatoes_dict.values())
flattened2 = [val for sublist in l2 for val in sublist]
dict2 = {}
for x in flattened2:
    if x in dict2:
        dict2[x]+= 1
    else:
        dict2[x] = 1
        
dict2_lower = {k.lower().strip(): v for k, v in dict2.items()}  
print(dict2_lower)

{'fantasy': 13, 'action': 10, 'adventure': 14, 'scifi': 5, 'horror': 4, 'mysteryandthriller': 7, 'kidsandfamily': 8, 'animation': 5, 'comedy': 18, 'drama': 21, 'musical': 2, 'crime': 5, 'music': 2, 'romance': 4, 'war': 2, 'history': 2}


In [24]:
#Counting occurrences of each genre of IMDB_dict and converting them into lowercase letters 
l3 = list(dictionary_movie_genre.values())
flattened3 = [val for sublist in l3 for val in sublist]
dict3 = {}
for x in flattened3:
    if x in dict3:
        dict3[x]+= 1
    else:
        dict3[x] = 1
        
dict3_lower = {k.lower().strip(): v for k, v in dict3.items()}  
print(dict3_lower)

{'drama': 29, 'crime': 4, 'action': 14, 'adventure': 2, 'biography': 5, 'history': 2, 'sci-fi': 6, 'romance': 2, 'western': 2, 'fantasy': 4, 'comedy': 2, 'thriller': 6, 'animation': 3, 'family': 2, 'war': 2, 'mystery': 5, 'music': 2, 'horror': 1}


In [25]:
#For each genre, if the corresponding genres are not present in the other two dictionaries, we are padding 0 (value) in front of them 
for key, value in dict1_lower.items():
    if key not in dict2_lower.keys():
        dict2_lower[key] = 0
    if key not in dict3_lower.keys():
        dict3_lower[key] = 0
        
for key, value in dict2_lower.items():
    if key not in dict1_lower.keys():
        dict1_lower[key] = 0
    if key not in dict3_lower.keys():
        dict3_lower[key] = 0
        
for key, value in dict3_lower.items():
    if key not in dict1_lower.keys():
        dict1_lower[key] = 0
    if key not in dict2_lower.keys():
        dict2_lower[key] = 0
        
        
print(dict1_lower)
print(dict2_lower)
print(dict3_lower)
print(len(dict1_lower))
print(len(dict2_lower))
print(len(dict3_lower))

{'drama': 39, 'mystery': 10, 'thriller': 13, 'crime': 4, 'romance': 11, 'war': 5, 'film-noir': 4, 'comedy': 9, 'musical': 3, 'history': 5, 'fantasy': 3, 'animation': 3, 'family': 3, 'adventure': 3, 'western': 2, 'action': 3, 'sport': 1, 'documentary': 1, 'biography': 2, 'horror': 3, 'music': 1, 'scifi': 0, 'mysteryandthriller': 0, 'kidsandfamily': 0, 'sci-fi': 0}
{'fantasy': 13, 'action': 10, 'adventure': 14, 'scifi': 5, 'horror': 4, 'mysteryandthriller': 7, 'kidsandfamily': 8, 'animation': 5, 'comedy': 18, 'drama': 21, 'musical': 2, 'crime': 5, 'music': 2, 'romance': 4, 'war': 2, 'history': 2, 'mystery': 0, 'thriller': 0, 'film-noir': 0, 'family': 0, 'western': 0, 'sport': 0, 'documentary': 0, 'biography': 0, 'sci-fi': 0}
{'drama': 29, 'crime': 4, 'action': 14, 'adventure': 2, 'biography': 5, 'history': 2, 'sci-fi': 6, 'romance': 2, 'western': 2, 'fantasy': 4, 'comedy': 2, 'thriller': 6, 'animation': 3, 'family': 2, 'war': 2, 'mystery': 5, 'music': 2, 'horror': 1, 'film-noir': 0, 'mus

In [26]:
#Now we are sorting the above such that the same genre lies in the same position and displaying the final sorted dictionary
dict1_sorted = OrderedDict(sorted(dict1_lower.items()))
dict2_sorted = OrderedDict(sorted(dict2_lower.items()))
dict3_sorted = OrderedDict(sorted(dict3_lower.items()))
print(dict1_sorted)
print(dict2_sorted)
print(dict3_sorted)

OrderedDict([('action', 3), ('adventure', 3), ('animation', 3), ('biography', 2), ('comedy', 9), ('crime', 4), ('documentary', 1), ('drama', 39), ('family', 3), ('fantasy', 3), ('film-noir', 4), ('history', 5), ('horror', 3), ('kidsandfamily', 0), ('music', 1), ('musical', 3), ('mystery', 10), ('mysteryandthriller', 0), ('romance', 11), ('sci-fi', 0), ('scifi', 0), ('sport', 1), ('thriller', 13), ('war', 5), ('western', 2)])
OrderedDict([('action', 10), ('adventure', 14), ('animation', 5), ('biography', 0), ('comedy', 18), ('crime', 5), ('documentary', 0), ('drama', 21), ('family', 0), ('fantasy', 13), ('film-noir', 0), ('history', 2), ('horror', 4), ('kidsandfamily', 8), ('music', 2), ('musical', 2), ('mystery', 0), ('mysteryandthriller', 7), ('romance', 4), ('sci-fi', 0), ('scifi', 5), ('sport', 0), ('thriller', 0), ('war', 2), ('western', 0)])
OrderedDict([('action', 14), ('adventure', 2), ('animation', 3), ('biography', 5), ('comedy', 2), ('crime', 4), ('documentary', 0), ('drama',

In [27]:
#For each final dictionary, we are creating a list of its values and displaying it 
h1 = list(dict1_sorted.values())
h2 = list(dict2_sorted.values())
h3 = list(dict3_sorted.values())
print(h1)
print(h2)
print(h3)

[3, 3, 3, 2, 9, 4, 1, 39, 3, 3, 4, 5, 3, 0, 1, 3, 10, 0, 11, 0, 0, 1, 13, 5, 2]
[10, 14, 5, 0, 18, 5, 0, 21, 0, 13, 0, 2, 4, 8, 2, 2, 0, 7, 4, 0, 5, 0, 0, 2, 0]
[14, 2, 3, 5, 2, 4, 0, 29, 2, 4, 0, 2, 1, 0, 2, 0, 5, 0, 2, 6, 0, 0, 6, 2, 2]


In [29]:
#Defining the cosine function and calculating it between all the above found lists h1, h2, h3
def cosinefunction(vec_a,vec_b):
    dot = sum(a*b for a, b in zip(vec_a, vec_b))
    norm_a = sum(a*a for a in vec_a) ** 0.5
    norm_b = sum(b*b for b in vec_b) ** 0.5
    cos_sim = dot / (norm_a*norm_b)
    return cos_sim
print(cosinefunction(h1,h2))
print(cosinefunction(h2,h3))
print(cosinefunction(h1,h3))

0.6824329998331107
0.6872637662866155
0.8780789215937731


In [31]:

#csv_columns = [Movie, Genres]
dict_data = list(metacritic_dict.items())
print(dict_data)
csv_file = "Metacritic_Genres.csv"
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, metacritic_dict.keys())
        writer.writeheader()
        #for data in dict_data:
            #writer.writerow(data)
        writer.writerow(metacritic_dict)
except IOError:
    print("I/O error")

[('citizen-kane', ['Drama', 'Mystery']), ('the-godfather', ['Drama', 'Thriller', 'Crime']), ('rear-window', ['Mystery', 'Thriller']), ('casablanca', ['Drama', 'Romance', 'War']), ('boyhood', ['Drama']), ('three-colors-red', ['Drama', 'Mystery', 'Romance']), ('vertigo', ['Mystery', 'Thriller', 'Romance']), ('notorious-1946', ['Drama', 'Thriller', 'Romance', 'Film-Noir']), ('singin-in-the-rain', ['Comedy', 'Romance', 'Musical']), ('city-lights', ['Drama', 'Comedy', 'Romance']), ('moonlight-2016', ['Drama']), ('intolerance', ['Drama', 'History']), ('pinocchio-1940', ['Fantasy', 'Comedy', 'Animation', 'Family', 'Musical']), ('touch-of-evil', ['Drama', 'Thriller', 'Crime', 'Film-Noir']), ('the-treasure-of-the-sierra-madre', ['Adventure', 'Drama', 'Western']), ('pans-labyrinth', ['Drama', 'Mystery', 'Thriller', 'Fantasy', 'War']), ('some-like-it-hot', ['Comedy', 'Romance']), ('north-by-northwest', ['Action', 'Adventure', 'Mystery', 'Thriller']), ('hoop-dreams', ['Drama', 'Sport', 'Documentar

In [ ]:
#csv_columns = [Movie, Genres]
dict_data = list(metacritic_dict.items())
print(dict_data)
csv_file = "Metacritic_Genres.csv"
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, metacritic_dict.keys())
        writer.writeheader()
        #for data in dict_data:
            #writer.writerow(data)
        writer.writerow(metacritic_dict)
except IOError:
    print("I/O error")

In [37]:
#csv_columns = [Movie, Genres]
dict_data = list(dictionary_movie_genre.items())
print(dict_data)
csv_file = "imdb_genres.csv"
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, dictionary_movie_genre.keys())
        writer.writeheader()
        #for data in dict_data:
            #writer.writerow(data)
        writer.writerow(dictionary_movie_genre)
except IOError:
    print("I/O error")

[('The Shawshank Redemption', ['Drama']), ('The Godfather', ['Crime', ' Drama']), ('The Dark Knight', ['Action', ' Crime', ' Drama']), ('The Godfather: Part II', ['Crime', ' Drama']), ('12 Angry Men', ['Crime', ' Drama']), ('The Lord of the Rings: The Return of the King', ['Action', ' Adventure', ' Drama']), ('Pulp Fiction', ['Crime', ' Drama']), ("Schindler's List", ['Biography', ' Drama', ' History']), ('Inception', ['Action', ' Adventure', ' Sci-Fi']), ('Fight Club', ['Drama']), ('The Lord of the Rings: The Fellowship of the Ring', ['Action', ' Adventure', ' Drama']), ('Forrest Gump', ['Drama', ' Romance']), ('The Good, the Bad and the Ugly', ['Western']), ('The Lord of the Rings: The Two Towers', ['Action', ' Adventure', ' Drama']), ('The Matrix', ['Action', ' Sci-Fi']), ('Goodfellas', ['Biography', ' Crime', ' Drama']), ('Star Wars: Episode V - The Empire Strikes Back', ['Action', ' Adventure', ' Fantasy']), ("One Flew Over the Cuckoo's Nest", ['Drama']), ('Hamilton', ['Biography'

In [36]:
#csv_columns = [Movie, Genres]
dict_data = list(rotten_tomatoes_dict.items())
print(dict_data)
csv_file = "Rotten_Tomatoes.csv"
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, rotten_tomatoes_dict.keys())
        writer.writeheader()
        #for data in dict_data:
            #writer.writerow(data)
        writer.writerow(rotten_tomatoes_dict)
except IOError:
    print("I/O error")

[('black_panther_2018', ['fantasy', 'action', 'adventure']), ('avengers_endgame', ['fantasy', 'adventure', 'scifi', 'action']), ('us_2019', ['horror', 'mysteryandthriller']), ('toy_story_4', ['fantasy', 'kidsandfamily', 'animation', 'adventure', 'comedy']), ('lady_bird', ['drama', 'comedy']), ('citizen_kane', ['drama']), ('mission_impossible_fallout', ['adventure', 'mysteryandthriller', 'action']), ('the_wizard_of_oz_1939', ['musical', 'kidsandfamily', 'fantasy']), ('blackkklansman', ['drama', 'comedy', 'crime']), ('get_out', ['horror', 'mysteryandthriller', 'comedy']), ('the_irishman', ['drama', 'crime']), ('mad_max_fury_road', ['adventure', 'action']), ('spider_man_into_the_spider_verse', ['fantasy', 'comedy', 'kidsandfamily', 'animation', 'adventure', 'action']), ('1003707-casablanca', ['drama']), ('moonlight_2016', ['drama']), ('a_star_is_born_2018', ['drama', 'music', 'romance']), ('wonder_woman_2017', ['adventure', 'action', 'fantasy']), ('dunkirk_2017', ['drama', 'war', 'history